# Merian project Part 4

**Find Jellyfish galaxies in Merian?**

In [1]:
%load_ext autoreload
%autoreload 2
import os, sys
sys.path.append(os.path.abspath('../../'))
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy.io import fits
from utils import pad_psf, show_image

# We can beautify our plots by changing the matpltlib setting a little
plt.rcParams['font.size'] = 18
plt.rcParams['image.origin'] = 'lower'
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['figure.dpi'] = 90
plt.rcParams['axes.linewidth'] = 2

- Download Yang group catalogs: https://gax.sjtu.edu.cn/data/DESI.html
- Cross-match with Merian in-band sources

column 1: group ID
column 2: richness
column 3: ra
column 4: dec
column 5: redshift
column 6: log M_h/(M_sun/h)
column 7: L_group (10^10Lsun/h/h) 

In [70]:
data = np.loadtxt('/Users/jiaxuanl/Downloads/DESIDR9_group/DESIDR9_NGC_group')
len(data)

48839245

In [71]:
cols = ['GroupId', 'richness', 'ra', 'dec', 'redshift', 'logMh/h', 'Lgroup']
cat = Table(data=data, names=cols)
cat = cat[(cat['redshift'] < 0.11) & (cat['redshift'] > 0.04)] # redshift selection

In [72]:
from utils import Rvir as compute_Rvir
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(70, 0.3, 0.7)
h = cosmo.h

In [73]:
cat['Rvir'] = compute_Rvir(10**(cat['logMh/h']) * h, z=cat['redshift']) # R200
cat['dist_ang_diam'] = cosmo.angular_diameter_distance(cat['redshift']).value
cat['Rvir_ang'] = np.rad2deg(cat['Rvir'] / cat['dist_ang_diam'] / 1000) # deg

In [78]:
np.max(cat['Rvir_ang'])

0.4528880301268168

In [75]:
cat.write('/Users/jiaxuanl/Downloads/DESIDR9_group/NGC_group_inband.fits')

In [127]:
def merian_footprint(ra, dec):
    if (ra > ((14 + 30/60) * u.hourangle).to(u.deg).value) & (ra < ((15 + 5/60) * u.hourangle).to(u.deg).value) & (dec < 0.6) & (dec > -2):
        return True
    elif (ra > ((11 + 30/60) * u.hourangle).to(u.deg).value) & (ra < ((12 + 35/60) * u.hourangle).to(u.deg).value) & (dec < 4.5) & (dec > 0):
        return True
    elif (ra > ((11 + 5/60) * u.hourangle).to(u.deg).value) & (ra < ((11 + 55/60) * u.hourangle).to(u.deg).value) & (dec < 0.5) & (dec > -2.5):
        return True
    elif (ra > ((9) * u.hourangle).to(u.deg).value) & (ra < ((10 + 50/60) * u.hourangle).to(u.deg).value) & (dec < 5.5) & (dec > -2.5):
        return True
    elif (ra > ((22 + 35/60) * u.hourangle).to(u.deg).value) & (ra < ((23 + 5/60) * u.hourangle).to(u.deg).value) & (dec < 4) & (dec > -0.5):
        return True
    elif (ra > ((23 + 30/60) * u.hourangle).to(u.deg).value) | (ra < ((12/60) * u.hourangle).to(u.deg).value) & (dec < 2.5) & (dec > -0.5):
        return True
    elif (ra > ((2 + 5/60) * u.hourangle).to(u.deg).value) | (ra < ((2 + 45/60) * u.hourangle).to(u.deg).value) & (dec < -2.7) & (dec > -7):
        return True
    else:
        return False

In [113]:
cat = Table.read('/Users/jiaxuanl/Downloads/DESIDR9_group/NGC_group_inband.fits')
cat = cat[(cat['dec'] > -3) & (cat['dec'] < 6)]
overlap = []
for obj in cat:
    overlap.append(merian_footprint(obj['ra'], obj['dec']))
cat['overlap_with_mer'] = overlap
cat = cat[cat['overlap_with_mer']]
cat.write('/Users/jiaxuanl/Downloads/DESIDR9_group/NGC_group_inband_overlap.fits')

In [128]:
cat = Table.read('/Users/jiaxuanl/Downloads/DESIDR9_group/SGC_group_inband.fits')
cat = cat[(cat['dec'] > -7) & (cat['dec'] < 3)]
overlap = []
for obj in cat:
    overlap.append(merian_footprint(obj['ra'], obj['dec']))
cat['overlap_with_mer'] = overlap
cat = cat[cat['overlap_with_mer']]
cat.write('/Users/jiaxuanl/Downloads/DESIDR9_group/SGC_group_inband_overlap.fits')

---

In [129]:
import astropy.units as u

In [153]:
mer_cat = Table.read('/Users/jiaxuanl/Downloads/Merian_DR1_photoz_EAZY_v2.0.fits')
mer_cat = mer_cat[mer_cat['z_spec'] > 0] # only spec-z sample
coords = np.array([mer_cat['coord_ra_Merian'], mer_cat['coord_dec_Merian']])

In [154]:
cat = Table.read('/Users/jiaxuanl/Downloads/DESIDR9_group/SGC_group_inband_overlap.fits')

In [156]:
from tqdm import trange

In [157]:
# A crude match
matched = []
for i in trange(len(cat)):
    host = cat[i]
    diff = np.sqrt((coords[0] - host['ra'])**2 + (coords[1] - host['dec'])**2)
    if np.sum(diff < 0.5):
        matched.append(True)
    else:
        matched.append(False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 94692/94692 [02:30<00:00, 627.65it/s]


In [158]:
cat['matched_crude'] = matched

In [162]:
cat.write('/Users/jiaxuanl/Downloads/DESIDR9_group/SGC_group_inband_overlap.fits', overwrite=True)

---

In [165]:
# Now we do a good match

In [196]:
cat = Table.read('/Users/jiaxuanl/Downloads/DESIDR9_group/SGC_group_inband_overlap.fits')
cat = cat[cat['matched_crude']]
host_coords = SkyCoord(cat['ra'], cat['dec'], unit='deg')

In [197]:
mer_cat = Table.read('/Users/jiaxuanl/Downloads/Merian_DR1_photoz_EAZY_v2.0.fits')
mer_cat = mer_cat[(mer_cat['z_spec'] > 0.05) & (mer_cat['z_spec'] < 0.11)] # only spec-z sample
coords = SkyCoord(mer_cat['coord_ra_Merian'], mer_cat['coord_dec_Merian'], unit='deg')

In [244]:
# A crude match
matched = []
for i in trange(len(cat[:3])):
    host = cat[i]
    host_coord = host_coords[i]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1105.90it/s]


In [245]:
temp = mer_cat[host_coord.separation(coords) < host['Rvir_ang'] * u.deg * 2]

In [246]:
host

GroupId,richness,ra,dec,redshift,logMh/h,Lgroup,Rvir,dist_ang_diam,Rvir_ang,overlap_with_mer,matched_crude
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,bool
20816.0,31.0,359.9805,0.7092,0.0824,13.7509,15.221,684.3423577943619,319.89126558479563,0.12257267722508362,True,True


In [247]:
temp = temp[np.abs(temp['z_spec'] - host['redshift']) < 0.005]

In [248]:
temp.sort('logssfr', reverse=True)

https://www.legacysurvey.org/viewer-merian/?ra=36.8502&dec=-4.6100&layer=merian-n708&zoom=14

https://www.legacysurvey.org/viewer-merian/?ra=36.5884&dec=-4.5815&layer=merian-n708&zoom=15&spectra

In [249]:
_coords = list(zip(temp['coord_ra_Merian'], temp['coord_dec_Merian']))

In [250]:
host

GroupId,richness,ra,dec,redshift,logMh/h,Lgroup,Rvir,dist_ang_diam,Rvir_ang,overlap_with_mer,matched_crude
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,bool
20816.0,31.0,359.9805,0.7092,0.0824,13.7509,15.221,684.3423577943619,319.89126558479563,0.12257267722508362,True,True


In [251]:
for item in temp:
    print(item['coord_ra_Merian'], item['coord_dec_Merian'])

359.96963915709904 0.7884811243229983
359.9000862798947 0.660487616064779
0.11925533981871615 0.8686651212212566
359.9406592755138 0.5612277874125536
359.91432437175825 0.5482277871904233
359.98129777617896 0.511763902086426
0.03854539303448809 0.4994679292812817
359.9504680821944 0.5437886286924508
0.05312900462464046 0.7792560809423572
0.05416378941876752 0.6652387823667287
359.9703258778549 0.5683135810528785
0.02007779501564437 0.7072678762340359
359.8552534498021 0.514607991298166
0.20231659291958623 0.7858998340468856
359.9948217512045 0.8029436691339366
359.8769118938474 0.6459623548255432
0.061487925905849165 0.6745439387543991
359.9351812299156 0.7311720428427038
359.96725347829 0.671611593331686
359.8548429206459 0.9101984641120517
359.93711073840063 0.7786508299203796
0.03536915522193219 0.7657932142663084
359.80642652661396 0.7288211017544747
0.02767487976674932 0.7451508772114019
359.9397399023481 0.6401669816347338
0.019795180628478404 0.7817126563084782
